## Data Merging

In [91]:
import pandas as pd
from difflib import SequenceMatcher

In [76]:
# Load CSV files into DataFrames
# df from Markets Business Insiders
aex_df = pd.read_csv('../data/raw_data/aex_stock_data.csv')
bel_20_df = pd.read_csv('../data/raw_data/bel_20_stock_data.csv')
cac_40_df = pd.read_csv('../data/raw_data/cac_40_stock_data.csv')
#iseq_20_df = pd.read_csv('../data/raw_data/iseq_20_stock_data.csv')
obx_df = pd.read_csv('../data/raw_data/obx_stock_data.csv')
osebx_df = pd.read_csv('../data/raw_data/osebx_stock_data.csv')
psi_20_df = pd.read_csv('../data/raw_data/psi_20_stock_data.csv')

# df from Wikipedia
aex_df_wiki = pd.read_csv('../data/raw_data/aex_wikipedia_data.csv')
bel_20_df_wiki = pd.read_csv('../data/raw_data/bel_20_wikipedia_data.csv')
cac_40_df_wiki = pd.read_csv('../data/raw_data/cac_40_wikipedia_data.csv')
#iseq_20_df_wiki = pd.read_csv('../data/raw_data/iseq_20_wikipedia_data.csv')
obx_df_wiki = pd.read_csv('../data/raw_data/obx_wikipedia_data.csv')
osebx_df_wiki = pd.read_csv('../data/raw_data/osebx_wikipedia_data.csv')
psi_20_df_wiki = pd.read_csv('../data/raw_data/psi_20_wikipedia_data.csv')

In [77]:
# Create the 'Index Name' column for each DataFrame
aex_df['Index'] = 'AEX'
bel_20_df['Index'] = 'BEL_20'
cac_40_df['Index'] = 'CAC_40'
#iseq_20_df['Index Name'] = 'ISEQ_20'
obx_df['Index'] = 'OBX'
osebx_df['Index'] = 'OSEBX'
psi_20_df['Index'] = 'PSI_20'

In [78]:
# Concatenate the DataFrames
df = pd.concat([aex_df, bel_20_df, cac_40_df, obx_df, osebx_df, psi_20_df])

# Reset the index to create a new unique primary key for each row
df.reset_index(drop=True, inplace=True)

# Select only the 'Index Name' and 'Name' columns
df = df[['Index', 'Name']]
df.rename(columns={'Name': 'Company'}, inplace=True)
# Display the first 40 rows of the resulting DataFrame
df

,Index,Company
0,AEX,ABN Amro
1,AEX,Adyen B.V. Parts Sociales
2,AEX,Ahold Delhaize
3,AEX,Akzo Nobel
4,AEX,ArcelorMittal
...,...,...
147,PSI_20,"REN - Redes Energeticas Nacionais SGPS, SAShs"
148,PSI_20,Sociedade de Investimento e Gestao SGPS SA SEMAPA
149,PSI_20,Sonae SGPS SA
150,PSI_20,Sonaecom SGPS SA


In [79]:
# Save the DataFrame to a CSV file in the 'data' folder
csv_filename = 'combined_stock_data.csv'
df.to_csv(f'../data/raw_data/{csv_filename}', index=False)

In [80]:
import re
import pandas as pd

# DataFrames list with their corresponding index names
df_to_clean = {
    'AEX': aex_df_wiki,
    'BEL_20': bel_20_df_wiki,
    'CAC_40': cac_40_df_wiki,
    'OBX': obx_df_wiki,
    'OSEBX': osebx_df_wiki,
    'PSI_20': psi_20_df_wiki
}

# Prepare the dataframes
for index_name, df in df_to_clean.items():
    # Standardize column names
    if 'Ticker symbol' in df.columns:
        df.rename(columns={'Ticker symbol': 'Ticker'}, inplace=True)
    if 'Name' in df.columns and index_name == 'OSEBX':
        df.rename(columns={'Name': 'Company'}, inplace=True)
    
    # Clean the 'Ticker' column
    df['Ticker'] = df['Ticker'].apply(lambda ticker: re.sub(r'.*:', '', ticker).strip())
    
    # Add the 'Index' column
    df['Index'] = index_name

# Combine all the prepared dataframes into one
df_wiki = pd.concat(df_to_clean.values())

# Select only the required columns and remove duplicates
df_wiki = df_wiki[['Index', 'Company', 'Ticker']].drop_duplicates().reset_index(drop=True)

# Function to clean company names
def clean_company_name(name):
    name = re.sub(r'\[.*?\]|\(.*?\)', '', name)
    name = name.replace('...', ' ')
    name = ' '.join(name.split())
    return name

# Apply the cleaning function to the 'Company' column
df_wiki['Company'] = df_wiki['Company'].apply(clean_company_name)

# Export the final dataframe to a CSV file
csv_filename = 'combined_wiki_data.csv'
df_wiki.to_csv(f'../data/raw_data/{csv_filename}', index=False)

# Display the first few rows of the final dataframe to verify
print(df_wiki.head(300))



      Index           Company Ticker
0       AEX             Adyen  ADYEN
1       AEX             Aegon    AGN
2       AEX    Ahold Delhaize     AD
3       AEX         AkzoNobel   AKZA
4       AEX     ArcelorMittal     MT
..      ...               ...    ...
295  PSI_20      Galp Energia   GALP
296  PSI_20           Ibersol    IBS
297  PSI_20  Jerónimo Martins    JMT
298  PSI_20        Mota-Engil    EGL
299  PSI_20               NOS    NOS

[300 rows x 3 columns]


In [93]:
# Function to find similarity between two strings
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

# Read data from both datasets
data1 = pd.read_csv('../data/raw_data/combined_wiki_data.csv')
data2 = pd.read_csv('../data/raw_data/combined_stock_data.csv')
combined_wiki_data = pd.read_csv('../data/raw_data/combined_wiki_data.csv')

company_mapping = {}

# Iterate over the companies in data1
for company1 in data1['Company']:
    max_similarity = 0
    mapped_company = ""
    for company2 in data2['Company']:
        similarity = similar(company1, company2)
        if similarity > max_similarity:
            max_similarity = similarity
            mapped_company = company2
    company_mapping[company1] = mapped_company
    
# Iterate over the companies in data2
for company2 in data2['Company']:
    max_similarity = 0
    mapped_company = ""
    for company1 in data1['Company']:
        similarity = similar(company1, company2)
        if similarity > max_similarity:
            max_similarity = similarity
            mapped_company = company1
    company_mapping[company2] = mapped_company
    

# Apply the mapping 
data1['Company'] = data1['Company'].map(company_mapping)
data2['Company'] = data2['Company'].map(company_mapping)

# Save the modified dataset to a new CSV file
#data1.to_csv('../data/raw_data/data1_mapped.csv', index=False)
#data2.to_csv('../data/raw_data/data2_mapped.csv', index=False)

# Merge the wiki data to get the tickers
combined_final_data = data2.merge(
    combined_wiki_data[['Company', 'Ticker']],
    on='Company',
    how='left'
)

missing_tickers = combined_final_data[combined_final_data['Ticker'].isna()]
missing_tickers.head()

combined_final_data = combined_final_data.drop_duplicates(subset=['Index', 'Company', 'Ticker'])

# Save the result to a CSV file
csv_filename = 'combined_final_data.csv'
combined_final_data.to_csv(f'../data/clean_data/{csv_filename}', index=False)
